# Ulepszanie wydajności klasyfikatorów

Wydajność modeli można zwiększać poprzez:

- dostrajanie parametrów pojedynczego modelu w celu jego ulepszenia. Do automatycznego dostrajania parametrów modelu w R można użyć pakietu `caret`. Pełna lista metod (algorytmów uczenia maszynowego) i ich dostosowywalnych parametrów znajduje się na [stronie internetowej](http://topepo.github.io/caret/available-models.html).

- łączenie ze sobą kilku słabych modeli w celu utworzenia lepszej jednostki. Jest to tak zwane uczenie zespołowe (ang. *ensemble learning*), któremu poświęcone są dzisiejsze ćwiczenia.

## Metody zespołowe

Podstawowe założenie: zaangażowanie zespołu różnych ekspertów (modeli) we wspólne rozwiązywanie problemu podniesie wydajność uczenia.

### Bagging

Jedna z najpopularniejszych technik uczenia zespołowego. Nazwa pochodzi od *bootstrap aggregating* - technika bagging wykorzystuje próbkowanie metodą bootstrap w funkcji alokacji do generowania danych przypisywanych do każdego z modeli w zespole. Zespoły w agregacji bootstrap zazwyczaj składają się z jednorodnych klasyfikatorów, które są trenowane równolegle i niezależnie od siebie.
Wśród metod bagging jedną z najpopularniejszych są *lasy losowe* (ang. *random forest*), składające się z wielu klasyfikatorów drzew decyzyjnych. Funkcja alokacji lasów łączy próbkowanie metodą bootstrap z losowym wybieraniem cech do generowania danych przydzielanych każdemu z klasyfikatorów w zespole. Lasy losowe nadają się do obsługiwania bardzo szerokich zbiorów danych.

### Boosting

Zespoły Wzmacniające (ang. *boosting*) są również jednorodnymi zbiorami podstawowych modeli. W odróżnieniu od agregacji bootstrap, we wzmacnianiu podstawowe modele są trenowane sekwencyjnie. Każdy kolejny model w sekwencji próbuje osiągnąć większą wydajność od poprzedniego modelu, ucząc się na błędach poprzednika. Następnie przeprowadzane jest wtórne próbkowanie danych treningowych tak, że przykładom dla których były nieprawidłowe predykcje, otrzymują większą wagę.

Proces wtórnego próbkowania, trenowania, oceniania i punktowania jest powtarzany do każdego modelu w sekwencji, dopóki wszystkie modele nie zostaną wytrenowane.

### Stacking

Stacking, czyli kontaminacja modeli, różni się od poprzednich dwóch tym, że zawiera tu zwykle różnorodne modele podstawowe. Przykładowo może składać się z modelu kNN, modelu regresji logistycznej oraz naiwnego modelu Bayesa. Kontaminacja modeli jest podobna do agregacji bootstrap, tyle że funkcja kombinacji jest tutaj niedeterministyczna (nie przebiega zgodnie z predefiniowanym wzorcem czy zbiorem reguł).

Zazponaj się z materiałami zamieszczonymi na stronie [RPubs](https://rpubs.com/mmazurek/346331), sekcje:

- Klasyfikatory złożone

- Agregacja bootstrapowa (bagging - bootstrap aggregation)

- Lasy losowe

- Boosting
  - Adaboost - adaptive boosting

Następnie wykonaj poniższe zadania.

## Zadania

1.	Pobierz i wczytaj w RStudio plik z danymi `Movie_classification.csv`. Celem zadania jest przewidzenie, czy dany film otrzyma Oskara czy też nie (zmienna dychotomiczna `Start_Tech_Oskar`).

In [1]:
library(tidyverse)
options(jupyter.rich_display = FALSE,
        repr.plot.width = 15,
        repr.plot.height = 8)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [24]:
df <- read_csv("data/Movie_classification.csv")
names(df) <- gsub(" ", "_", names(df))
names(df)[names(df) == '3D_available'] <- 'is_3D_available'

Rows: 506 Columns: 19
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): 3D_available, Genre
dbl (17): Marketing expense, Production expense, Multiplex coverage, Budget,...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


2.	Przyglądnij się danym i przygotuj je do dalszej analizy:

 - braki danych zastąp wartością średnią dla danego atrybutu, 
 - za pomocą funkcji `sample.split` (z pakietu `caTools`) oraz `subset` podziel dane na zbiór treningowy i testowy.

In [25]:
names(df)

 [1] "Marketing_expense"   "Production_expense"  "Multiplex_coverage" 
 [4] "Budget"              "Movie_length"        "Lead__Actor_Rating" 
 [7] "Lead_Actress_rating" "Director_rating"     "Producer_rating"    
[10] "Critic_rating"       "Trailer_views"       "is_3D_available"    
[13] "Time_taken"          "Twitter_hastags"     "Genre"              
[16] "Avg_age_actors"      "Num_multiplex"       "Collection"         
[19] "Start_Tech_Oscar"   

In [26]:
dim(df)

[1] 506  19

In [27]:
glimpse(df)

Rows: 506
Columns: 19
$ Marketing_expense   <dbl> 20.1264, 20.5462, 20.5458, 20.6474, 21.3810, 20.59…
$ Production_expense  <dbl> 59.62, 69.14, 69.14, 59.36, 59.36, 59.36, 70.74, 7…
$ Multiplex_coverage  <dbl> 0.462, 0.531, 0.531, 0.542, 0.542, 0.542, 0.476, 0…
$ Budget              <dbl> 36524.12, 35668.65, 39912.68, 38873.89, 39701.58, …
$ Movie_length        <dbl> 138.7, 152.4, 134.6, 119.3, 127.7, 132.2, 140.1, 1…
$ Lead__Actor_Rating  <dbl> 7.825, 7.505, 7.485, 6.895, 6.920, 6.890, 7.065, 6…
$ Lead_Actress_rating <dbl> 8.095, 7.650, 7.570, 7.035, 7.070, 7.100, 7.265, 7…
$ Director_rating     <dbl> 7.910, 7.440, 7.495, 6.920, 6.815, 6.885, 7.150, 6…
$ Producer_rating     <dbl> 7.995, 7.470, 7.515, 7.020, 7.070, 7.005, 7.400, 7…
$ Critic_rating       <dbl> 7.94, 7.44, 7.44, 8.26, 8.26, 7.26, 8.96, 7.96, 7.…
$ Trailer_views       <dbl> 527367, 494055, 547051, 516279, 531448, 498425, 45…
$ is_3D_available     <chr> "YES", "NO", "NO", "YES", "NO", "YES", "YES", "NO"…
$ Time_taken      

In [19]:
summary(df)

 Marketing_expense Production_expense Multiplex_coverage     Budget     
 Min.   :  20.13   Min.   : 55.92     Min.   :0.1290     Min.   :19781  
 1st Qu.:  21.64   1st Qu.: 65.38     1st Qu.:0.3760     1st Qu.:32694  
 Median :  25.13   Median : 74.38     Median :0.4620     Median :34488  
 Mean   :  92.27   Mean   : 77.27     Mean   :0.4453     Mean   :34911  
 3rd Qu.:  93.54   3rd Qu.: 91.20     3rd Qu.:0.5510     3rd Qu.:36794  
 Max.   :1799.52   Max.   :110.48     Max.   :0.6150     Max.   :48773  
                                                                        
  Movie_length   Lead__Actor_Rating Lead_Actress_rating Director_rating
 Min.   : 76.4   Min.   :3.840      Min.   :4.035       Min.   :3.840  
 1st Qu.:118.5   1st Qu.:7.316      1st Qu.:7.504       1st Qu.:7.296  
 Median :151.0   Median :8.307      Median :8.495       Median :8.312  
 Mean   :142.1   Mean   :8.014      Mean   :8.186       Mean   :8.020  
 3rd Qu.:167.6   3rd Qu.:8.865      3rd Qu.:9.030       

In [28]:
df <- df %>%
  mutate(
    Trailer_views = as.integer(Trailer_views),
    is_3D_available = is_3D_available == "YES",
    Genre = as.factor(Genre),
    Avg_age_actors = as.integer(Avg_age_actors),
    Num_multiplex = as.integer(Num_multiplex),
    Collection = as.integer(Collection),
    Start_Tech_Oscar = as.factor(Start_Tech_Oscar),
  )

In [29]:
df %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()

[1] 12

In [30]:
sapply(df, function(x) sum(length(which(is.na(x)))))

  Marketing_expense  Production_expense  Multiplex_coverage              Budget 
                  0                   0                   0                   0 
       Movie_length  Lead__Actor_Rating Lead_Actress_rating     Director_rating 
                  0                   0                   0                   0 
    Producer_rating       Critic_rating       Trailer_views     is_3D_available 
                  0                   0                   0                   0 
         Time_taken     Twitter_hastags               Genre      Avg_age_actors 
                 12                   0                   0                   0 
      Num_multiplex          Collection    Start_Tech_Oscar 
                  0                   0                   0 

In [31]:
df["Time_taken"][is.na(df["Time_taken"])] <- as.integer(mean(df$Time_taken, na.rm = TRUE))

In [32]:
df %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()

[1] 0

3. Dokonaj klasyfikacji za pomocą trzech algorytmów złożonych: bagging, random forest oraz boosting (zbuduj modele na zbiorze treningowym).

4. Sprawdź jak dobrze utworzone modele radzą sobie z danymi testowymi (użyj funkcji `confusionMatrix` z pakietu `caret`). Jaki procent danych został poprawnie zaklasyfikowany? Który model okazał się w tym przypadku najlepszy?